In [1]:
import time
s = time.time()

In [2]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [4]:
df = pd.merge(listings, reviews, on='listing_id')

In [5]:
df.describe();

In [6]:
df.head();

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments']))

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['lisa', 'is', 'wonderful', 'kind', 'and', 'thoughtful', 'host', 'the', 'listing', 'is', 'accurate', 'there', 'is', 'an', 'entire', 'floor', 'for', 'the', 'guest', 'with', 'full', 'bath', 'and', 'kitchen', 'area', 'looking', 'out', 'over', 'leafy', 'backyard', 'everything', 'needed', 'for', 'my', 'one', 'month', 'stay']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

In [12]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [13]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['wonderful', 'kind', 'thoughtful', 'host', 'list', 'accurate', 'entire', 'floor', 'guest', 'full', 'area', 'look', 'backyard', 'need', 'month', 'stay']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"time" + 0.055*"amazing" + 0.052*"thank" + 0.048*"go" + 0.041*"love" '
  '+ 0.040*"come" + 0.038*"experience" + 0.037*"back" + 0.032*"much" + '
  '0.031*"city"'),
 (1,
  '0.197*"host" + 0.196*"clean" + 0.080*"need" + 0.058*"well" + 0.039*"day" + '
  '0.038*"helpful" + 0.038*"friendly" + 0.030*"accommodate" + 0.028*"arrival" '
  '+ 0.023*"cancel"'),
 (2,
  '0.124*"good" + 0.123*"nice" + 0.089*"really" + 0.060*"also" + 0.058*"room" '
  '+ 0.052*"area" + 0.041*"lot" + 0.036*"enjoy" + 0.027*"price" + '
  '0.020*"give"'),
 (3,
  '0.256*"stay" + 0.152*"apartment" + 0.088*"definitely" + 0.080*"easy" + '
  '0.065*"super" + 0.037*"quiet" + 0.035*"convenient" + 0.035*"locate" + '
  '0.034*"spacious" + 0.021*"access"'),
 (4,
  '0.096*"comfortable" + 0.080*"stay" + 0.077*"space" + 0.072*"make" + '
  '0.057*"feel" + 0.052*"home" + 0.038*"beautiful" + 0.034*"wonderful" + '
  '0.026*"lovely" + 0.026*"family"'),
 (5,
  '0.477*"place" + 0.049*"look" + 0.043*"responsive" + 0.037*"excellent

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.9453713903215

Coherence Score:  0.6079556732309548


In [18]:
print((s-time.time())/60/60)

-4.073758807513449


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.094311  0.117944       1        1  12.231339
7      0.145565 -0.263499       2        1  12.168573
9      0.062973  0.035626       3        1  10.855184
0      0.055946  0.064631       4        1  10.131586
4     -0.328643 -0.058767       5        1   9.806294
2      0.141622 -0.291980       6        1   9.671734
3     -0.338386 -0.052688       7        1   9.502460
1      0.071228  0.126881       8        1   8.941132
6      0.058318  0.217142       9        1   8.858697
5      0.037067  0.104708      10        1   7.833008, topic_info=     Category           Freq      Term          Total  loglift  logprob
34    Default  833312.000000     place  833312.000000  30.0000  30.0000
60    Default  569495.000000     great  569495.000000  29.0000  29.0000
13    Default  717724.000000      stay  717724.000000  28.0000  28.0000
7     Default  392066.000000      host  392066.000000  27.0000  27.0000
22    Default  391248.000000     clean  391248.000000  26.0000  26.0000
...       ...            ...       ...            ...      ...      ...
1467  Topic10    5187.701172  strongly    5188.809082   2.5466  -5.8197
854   Topic10    4824.182129    amable    4825.290039   2.5466  -5.8924
278   Topic10   10470.311523   service   11511.510742   2.4520  -5.1175
271   Topic10    8991.351562   message   22906.468750   1.6117  -5.2697
47    Topic10    6893.646973       top   11501.552734   2.0349  -5.5354

[373 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
123       1  0.999958         able
479       4  0.999980   absolutely
16        7  0.999967       access
246      10  0.999918   accessible
17        8  0.999977  accommodate
...     ...       ...          ...
1074      2  0.999950       window
15        5  0.999991    wonderful
368       2  0.999980         work
1075      6  0.999939        worth
73        9  0.999996        would

[377 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 10, 1, 5, 3, 4, 2, 7, 6])